# Policy Checker with Azure AI Search

This notebook demonstrates how to use agent operations with the Azure AI Search tool from the Azure Agents service using a synchronous client.

## Setup and Imports

In [1]:
# Import necessary libraries and modules
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool, ListSortOrder, MessageRole

# Load environment variables from .env file
load_dotenv(override=True)  # This forces a reload of the .env file

print("✅ Libraries imported and environment variables loaded")

✅ Libraries imported and environment variables loaded


## Configuration

In [8]:
# Retrieve endpoint and model deployment name from environment variables
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")  # Ensure the PROJECT_ENDPOINT environment variable is set
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")  # Ensure the MODEL_DEPLOYMENT_NAME environment variable is set
sc_connection_id = os.environ.get("AZURE_AI_CONNECTION_ID")  # Ensure the AZURE_AI_CONNECTION_ID environment variable is set

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False),  # Use Azure Default Credential for authentication
)

with project_client:
    # Initialize the Azure AI Search tool with the required parameters
    ai_search = AzureAISearchTool(
        index_connection_id=sc_connection_id,  # Ensure the AZURE_AI_CONNECTION_ID environment variable is set
        index_name="insurance-documents-index",  # Name of the search index
        query_type=AzureAISearchQueryType.SIMPLE,  # Query type (e.g., SIMPLE, FULL)
        top_k=3,  # Number of top results to retrieve
        filter="",  # Optional filter for search results
    )

    # Create an agent with the specified model, name, instructions, and tools
    agent = project_client.agents.create_agent(
        model=model_deployment_name,  # Model deployment name
        name="policy-checker",  # Name of the agent
        instructions=
        """"
        You are an expert Insurance Policy Checker Agent specialized in analyzing auto insurance policies and validating claim coverage. Your primary responsibilities include:

        **Core Functions:**
        - Analyze insurance policy documents to determine coverage details
        - Validate if specific claims are covered under policy terms
        - Explain policy limits, deductibles, and exclusions
        - Identify coverage gaps or restrictions
        - Provide clear explanations of policy benefits

        **Policy Types You Handle:**
        - Commercial Auto Policies
        - Comprehensive Auto Policies  
        - High Value Vehicle Policies
        - Liability Only Policies
        - Motorcycle Policies

        **Analysis Guidelines:**
        1. Always reference specific policy sections when providing coverage determinations
        2. Clearly state coverage limits, deductibles, and any applicable restrictions
        3. Identify any exclusions that may apply to the claim
        4. Be precise about effective dates and policy periods
        5. Flag any discrepancies between claim details and policy terms

        **Response Format:**
        - Start with a clear coverage determination (COVERED/NOT COVERED/PARTIAL COVERAGE)
        - Provide the specific policy section reference
        - Explain coverage limits and deductibles
        - List any relevant exclusions or conditions
        - Suggest next steps if coverage issues exist
        - Everything in a clear, concise manner in one paragraph.

        **Tone:** Professional, accurate, and helpful. Always be thorough in your analysis while remaining clear and concise.

        When you cannot find specific information in the policy documents, clearly state what information is missing and what additional documentation would be needed.
        """,  # Instructions for the agent
        tools=ai_search.definitions,  # Tools available to the agent
        tool_resources=ai_search.resources,  # Resources for the tools
    )
    print(f"Created agent, ID: {agent.id}")

    # Create a thread for communication with the agent
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    # Send a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread.id,  # ID of the thread
        role=MessageRole.USER,  # Role of the message sender (e.g., user)
        content="What are the most comprehensive insurance policies for cars?",  # Message content
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run in the thread using the tools
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Log the error if the run fails
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages from the thread
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        # Only show the agent's response, not the user's question
        if message.role == MessageRole.AGENT:
            if message.content and len(message.content) > 0:
                content_item = message.content[0]
                if content_item.get('type') == 'text' and 'text' in content_item:
                    agent_response = content_item['text']['value']
                    print(agent_response)
                    break  # Only show the first agent response

Created agent, ID: asst_mkx5KyubypWBuzDiaYs2ShEd
Created thread, ID: thread_MnHil91IvJj9Zs1pEJ3ik8Go
Created thread, ID: thread_MnHil91IvJj9Zs1pEJ3ik8Go
Created message, ID: msg_BeIPqSobHnqZ4X0UidtySo6i
Created message, ID: msg_BeIPqSobHnqZ4X0UidtySo6i
Run finished with status: RunStatus.COMPLETED
COVERED: The most comprehensive insurance policies for cars are typically Comprehensive Auto Policies. These policies provide broad coverage including physical damage to your vehicle from non-collision events such as theft, fire, vandalism, natural disasters, and falling objects, in addition to collision coverage for accidents. Such policies usually include liability coverage for bodily injury and property damage, medical payments, uninsured motorist protection, and often roadside assistance. Coverage limits and deductibles vary by policy but typically are customizable. Relevant sections to review are "Comprehensive Coverage" and "Collision Coverage" in the policy document. Be aware of exclus

Amazing! So what you have done so far is creating an agent, that consults the Azure AI Search Index that searches on your policies databases. Your query has made it such as it replies to your query with a text based solution. Let's print it out!

In [10]:
# Cell 1: Create the agent and print/save its ID
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import AzureAISearchQueryType, AzureAISearchTool

load_dotenv(override=True)
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
sc_connection_id = os.environ.get("AZURE_AI_CONNECTION_ID")

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False),
)

with project_client:
    ai_search = AzureAISearchTool(
        index_connection_id=sc_connection_id,
        index_name="insurance-documents-index",
        query_type=AzureAISearchQueryType.SIMPLE,
        top_k=3,
        filter="",
    )
    agent = project_client.agents.create_agent(
        model=model_deployment_name,
        name="policy-checker",
        instructions="""
        You are an expert Insurance Policy Checker Agent specialized in analyzing auto insurance policies and validating claim coverage. Your primary responsibilities include: ... (instructions truncated) ...
        """,
        tools=ai_search.definitions,
        tool_resources=ai_search.resources,
    )
    agent_id = agent.id
    print(f"Created agent, ID: {agent_id}")
# Save agent_id for use in other cells

Created agent, ID: asst_NYqE54ktEPxYEYNS4rKTZXnB


In [11]:
# Cell 2: Use the agent ID in a new session/cell for queries
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ListSortOrder, MessageRole

load_dotenv(override=True)
project_endpoint = os.environ.get("AI_FOUNDRY_PROJECT_ENDPOINT")
agent_id = "asst_NYqE54ktEPxYEYNS4rKTZXnB"  # Replace with the agent_id printed from the previous cell

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(exclude_interactive_browser_credential=False),
)

user_query = "What are the most comprehensive insurance policies for cars?"

with project_client:
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=user_query,
    )
    print(f"Created message, ID: {message['id']}")
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent_id)
    print(f"Run finished with status: {run.status}")
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    messages = project_client.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
    for message in messages:
        if message.role == MessageRole.AGENT:
            if message.content and len(message.content) > 0:
                content_item = message.content[0]
                if content_item.get('type') == 'text' and 'text' in content_item:
                    agent_response = content_item['text']['value']
                    print(agent_response)
                    break

Created thread, ID: thread_M2kPUqYgnwSFzYQ1U8xmfHxr
Created message, ID: msg_Tmb6G2KTCu0PPhmJldyy0ukz
Created message, ID: msg_Tmb6G2KTCu0PPhmJldyy0ukz
Run finished with status: RunStatus.COMPLETED
Run finished with status: RunStatus.COMPLETED
The most comprehensive car insurance policies typically include the following coverages:

1. **Liability Coverage**: Covers bodily injury and property damage that you cause to others.
2. **Collision Coverage**: Pays for damage to your car resulting from a collision with another vehicle or object.
3. **Comprehensive Coverage**: Covers damage to your car from non-collision events such as theft, fire, vandalism, natural disasters, and falling objects.
4. **Personal Injury Protection (PIP)** or **Medical Payments Coverage**: Covers medical expenses for you and your passengers regardless of fault.
5. **Uninsured/Underinsured Motorist Coverage**: Protects you if you're hit by a driver who has no insurance or insufficient insurance.
6. **Rental Reimburs